In [1]:
import numpy as np
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys

import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
from pandas import DataFrame
from pandas.core.dtypes import common as com
from core.loader import Loader
from models.model_wrapper import ModelWrapper
from core.validator import ModelValidator, load_saved_split, load_train_split

from tensorflow.python.client import device_lib

for device in device_lib.list_local_devices():
    print(device.physical_device_desc)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# TODO: Zkusit 3 stupně na NN, SVM a porovnat s LightGBM + XGBoost

# 4 x Malware
# 4x  Phishing
# + DGA (jeden stupen)

2025-05-13 12:12:13.550607: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-13 12:12:13.550646: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-13 12:12:13.588893: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-13 12:12:13.668296: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-13 12:12:14.646440: W tensorflow/compiler/tf2

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
2025-05-13 12:12:18.665352: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2025-05-13 12:12:18.665376: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: poli
2025-05-13 12:12:18.665381: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: poli
2025-05-13 12:12:18.665480: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 570.133.7
2025-05-13 12:12:18.665500: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module 

In [ ]:
model_wrapper = ModelWrapper(model_dir="models")
from core.validator import ModelValidator, load_saved_split, load_train_split


# ARCH_NAME = "cnn" 
#ARCH_NAME = "XgBoost" 
#ARCH_NAME = "Lgbm"
ARCH_NAME = "attention_tls"
VERSION = "v1.1"
#VERSION = "v1.0"
malicious_label = "phishing"
stage = 3
verification = True


#prefix=f"tls_agg"
prefix=f"stage_{stage}"

model = model_wrapper.load(arch_name=ARCH_NAME, label=malicious_label, prefix=prefix, version=VERSION)

x_test, y_test = load_saved_split(stage, label=malicious_label, folder="./data/", verification=verification)



# Initialize validator class
validator = ModelValidator(model, x_test, y_test, arch_name=ARCH_NAME, label=malicious_label, prefix=f"stage_{stage}", version=VERSION, verification=verification, stage=stage)  



validator.evaluate_performance(save_results = True)


📦 Loading model from models/attention_tls_stage_3_malware_v1.1.keras

🔍 Starting model evaluation...
X_test shape: (370, 176)
Scaled and selected attention_tls input features
12/12 [==============================] - 0s 970us/step

📊 Saving evaluation metrics to .tex and CF matrix

📋 Evaluation Summary:
+---------------------------+----------+
| Metric                    |    Value |
+===========================+==========+
| Accuracy                  |   0.4892 |
+---------------------------+----------+
| Precision                 |   0.1058 |
+---------------------------+----------+
| Recall                    |   0.8800 |
+---------------------------+----------+
| F1 Score                  |   0.1888 |
+---------------------------+----------+
| ROC AUC                   |   0.6704 |
+---------------------------+----------+
| True Negatives (TN)       | 159.0000 |
+---------------------------+----------+
| False Positives (FP)      | 186.0000 |
+---------------------------+----------+

X does not have valid feature names, but StandardScaler was fitted with feature names


🖼️ Confusion matrix plot saved to: confusion/attention_tls_stage_3_malware_v1.1_verification_confusion_matrix.png


📝 Classification Report:
              precision    recall  f1-score   support

           0   0.981481  0.460870  0.627219       345
           1   0.105769  0.880000  0.188841        25

    accuracy                       0.489189       370
   macro avg   0.543625  0.670435  0.408030       370
weighted avg   0.922312  0.489189  0.597599       370



## Run Verification / Validation for all models 
- Generates Confusion Matrices
- Generates Classification Reports
- Generates .TEX table and all the measurement data 

In [ ]:
model_wrapper = ModelWrapper(model_dir="models")
architectures = ["XgBoost"]

# , "XgBoost", "Lgbm"

malicious_labels = ["phishing"]
stages = [1, 2, 3]
VERSION = "v1.1"
verification = False

for architecture in architectures:       
    for malicious_label in malicious_labels:
        for stage in stages:
            model = model_wrapper.load(arch_name=architecture, label=malicious_label, prefix=f"stage_{stage}", version=VERSION)
            x_test, y_test = load_saved_split(stage, malicious_label, folder="./data/", verification=verification)
            
            # print number of columns for x_test
            print(f"Number of columns in x_test: {x_test.shape[1]}")
            
            # # take only the first 1000 samples
            # x_test = x_test[:1000]
            # y_test = y_test[:1000]
            
            # validator = ModelValidator(model, x_test, y_test, arch_name=architecture, label=malicious_label, prefix=f"stage_{stage}", version=VERSION, verification=verification, stage=stage)  
            # validator.evaluate_performance()
            